# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet'])

import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict, RandomizedSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/samuelezebunandu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/samuelezebunandu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///../DisasterResponse.db')
df = pd.read_sql('SELECT * FROM messages', engine)
X = df['message']
y = df.drop(['id','message', 'original', 'genre'], axis=1)

In [3]:
print(X.shape, y.shape)

(26216,) (26216, 36)


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.isalpha()]
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token).strip()
        clean_tokens.append(clean_token)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('text_pipeline', Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer())
    ])),
    ('clf', RandomForestClassifier(random_state=42))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [8]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('text_pipeline',
                 Pipeline(memory=None,
                          steps=[('vect',
                                  CountVectorizer(analyzer='word', binary=False,
                                                  decode_error='strict',
                                                  dtype=<class 'numpy.int64'>,
                                                  encoding='utf-8',
                                                  input='content',
                                                  lowercase=True, max_df=1.0,
                                                  max_features=None, min_df=1,
                                                  ngram_range=(1, 1),
                                                  preprocessor=None,
                                                  stop_words=None,
                                                  strip_accents=None,
                                                  token_patter

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_train_pred = cross_val_predict(pipeline, X_train, y_train, cv=3)

In [10]:
from sklearn.metrics import classification_report
y_train_pred_df = pd.DataFrame(y_train_pred)
labels = y_test.columns.to_list()
y_pred_df = pd.DataFrame(y_train_pred)
for i in range(y_train_pred.shape[1]):
    print(f'\033[1m{labels[i]}\033[0m')
    print(classification_report(y_train.iloc[:, i], y_train_pred_df.iloc[:, i]), end='')
    print('------------------------------------------------------------')
    print()

related
              precision    recall  f1-score   support

           0       0.71      0.38      0.50      4615
           1       0.83      0.95      0.89     14904
           2       0.49      0.49      0.49       143

    accuracy                           0.81     19662
   macro avg       0.68      0.61      0.62     19662
weighted avg       0.80      0.81      0.79     19662
------------------------------------------------------------

request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94     16317
           1       0.89      0.42      0.57      3345

    accuracy                           0.89     19662
   macro avg       0.89      0.70      0.75     19662
weighted avg       0.89      0.89      0.88     19662
------------------------------------------------------------

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19569
           1       0.00      0.00      

/Users/samuelezebunandu/Documents/Courses/Udacity/DSND/DSND_Projects/Disaster REsponse Pipelines/.env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19662

    accuracy                           1.00     19662
   macro avg       1.00      1.00      1.00     19662
weighted avg       1.00      1.00      1.00     19662
------------------------------------------------------------

water
              precision    recall  f1-score   support

           0       0.95      1.00      0.97     18421
           1       0.94      0.15      0.26      1241

    accuracy                           0.95     19662
   macro avg       0.94      0.58      0.62     19662
weighted avg       0.95      0.95      0.93     19662
------------------------------------------------------------

food
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     17493
           1       0.89      0.29      0.44      2169

    accuracy                           0.92     19662
   macro avg       0.90      0.64      0.70     1

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     17856
           1       0.80      0.17      0.28      1806

    accuracy                           0.92     19662
   macro avg       0.86      0.58      0.62     19662
weighted avg       0.91      0.92      0.90     19662
------------------------------------------------------------

fire
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     19448
           1       0.00      0.00      0.00       214

    accuracy                           0.99     19662
   macro avg       0.49      0.50      0.50     19662
weighted avg       0.98      0.99      0.98     19662
------------------------------------------------------------

earthquake
              precision    recall  f1-score   support

           0       0.95      0.99      0.97     17806
           1       0.90      0.50      0.64      1856

    accuracy                           0.95 

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline = Pipeline([
    ('text_pipeline', Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer())
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
])

In [12]:
parameters = {
    'text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    'text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
    'text_pipeline__vect__max_features': (None, 5000, 10000),
    'text_pipeline__tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__min_samples_split': [2, 3, 4],
}

rcv = RandomizedSearchCV(pipeline, param_distributions=parameters, n_jobs=-1)

In [ ]:
rcv.fit(X_train, y_train)

In [ ]:
rcv.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_test_pred = rcv.predict(X_test)

In [ ]:
y_test_pred_df = pd.DataFrame(y_test_pred)
labels = y_test.columns.to_list()
for i in range(y_train_pred.shape[1]):
    print(f'\033[1m{labels[i]}\033[0m')
    print(classification_report(y_test.iloc[:, i], y_test_pred_df.iloc[:, i]), end='')
    print('------------------------------------------------------------')
    print()

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
with open('ml_pipeline.pkl', 'wb') as m:
    pickle.dump(rcv, m)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.